<a href="https://colab.research.google.com/github/Wenuka19/DermAI/blob/main/Colab/DermAI_MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
!pip install keras-vis
!pip install tf.keras-vis tensorflow

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from IPython.display import clear_output 
import wandb
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

from wandb.keras import WandbCallback

from keras.models import Model
from keras.optimizers import Adam
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten,GlobalAveragePooling2D
from pathlib import Path
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator

from tf_keras_vis.saliency import Saliency
from vis.utils import utils

import seaborn as sns 
import pandas as pd
import sklearn.metrics

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
# your rest of the code

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# This is my private API key for WandB login. So don't share this anywhere 🥲
!wandb login 0150b3d8e70b00c4b8b62cbf7c729ea59d224292

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [ ]:
# # # # For toy model
# IMG_SIZE = (256,256)

IMG_SIZE = (224,224)

In [ ]:
#copy data to memory. upload it as a zip file. create a folder called 'data' before executing

!cp '/content/drive/MyDrive/DermAI_Datasets/fitzpatrick17k/output.zip' '/content/data'

In [ ]:
train_data_dir= os.path.join('/content','data','output','train')
val_data_dir = os.path.join('/content','data','output','val')
test_data_dir = os.path.join('/content','data','output','test')

In [ ]:
# !unzip '/content/data/dataset.zip'
!unzip '/content/data/output.zip' -d '/content/data'
clear_output()
!rm '/content/data/output.zip'

In [ ]:
#Load data as numpy arrays
train_data = tf.keras.utils.image_dataset_from_directory(train_data_dir,label_mode='categorical',  image_size=IMG_SIZE,batch_size=32)
val_data = tf.keras.utils.image_dataset_from_directory(val_data_dir,label_mode='categorical',  image_size=IMG_SIZE,batch_size=32)
test_data = tf.keras.utils.image_dataset_from_directory(test_data_dir,label_mode='categorical',  image_size=IMG_SIZE,batch_size=32)

Found 13212 files belonging to 114 classes.
Found 1612 files belonging to 114 classes.
Found 1753 files belonging to 114 classes.


In [ ]:
train_dataset = train_data.map(lambda x,y:(preprocess_input(x),y))
val_dataset = val_data.map(lambda x,y:(preprocess_input(x),y))
test_dataset = test_data.map(lambda x,y:(preprocess_input(x),y))

In [ ]:
class_names = train_data.class_names

In [ ]:
labels = '\n'.join(sorted(class_names))

with open('labels.txt', 'w') as f:
  f.write(labels)

In [ ]:
def create_model(input_shape, n_classes, optimizer='adam', fine_tune=0):
    conv_base = MobileNet(input_shape=input_shape, include_top=False,weights=None)
    
    for layer in conv_base.layers:
      layer.trainable=True

    x = conv_base.output
    x=GlobalAveragePooling2D()(x)
    x=Dense(1024,activation='relu')(x) 
    x=Dense(512,activation='relu')(x)
    preds=Dense(n_classes,activation='softmax')(x)
    model = Model(inputs=conv_base.input,outputs=preds)
    
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
IMG_SHAPE = IMG_SIZE + (3,)
n_classes = len(class_names)

In [ ]:
base_learning_rate = 0.01
optim_1 = RMS(learning_rate=base_learning_rate)
mobilenet_model = create_model(IMG_SHAPE, n_classes, optim_1, fine_tune=0)

In [ ]:
mobilenet_model.optimizer.get_config()

{'name': 'Adam',
 'learning_rate': 0.01,
 'decay': 0.0,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'epsilon': 1e-07,
 'amsgrad': False}

In [ ]:
mobilenet_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [ ]:
initial_epochs = 25

In [ ]:
fine_tune_from=80
for layer in mobilenet_model.layers[fine_tune_from:]:
  layer.trainable = True

In [ ]:
for i in mobilenet_model.layers:
  print(i.trainable)

In [ ]:
config = {"Name":"MobileNet","lr": 0.01, "batch_size": 32,"new_model":True}

wandb.init(project="DermAI-01", save_code=True,config=config)
callback =  tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)
history = mobilenet_model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=val_dataset,callbacks=[WandbCallback(save_model=False),callback])

accuracy,▁▃▄▆███
epoch,▁▂▃▅▆▇█
loss,█▄▃▃▂▁▁
val_accuracy,▃▁▅█▇▂▆
val_loss,█▁▁▁▄▄▂
accuracy,0.06312
best_epoch,3
best_val_loss,4.55702
epoch,6
loss,4.23664
val_accuracy,0.04529


Epoch 1/25
  6/413 [..............................] - ETA: 1:19 - loss: 13.4201 - accuracy: 0.0208

413/413 [==============================] - 91s 211ms/step - loss: 4.6768 - accuracy: 0.0366 - val_loss: 4.5057 - val_accuracy: 0.0285
Epoch 2/25
413/413 [==============================] - 88s 211ms/step - loss: 4.5129 - accuracy: 0.0399 - val_loss: 4.5610 - val_accuracy: 0.0391
Epoch 3/25
413/413 [==============================] - 88s 210ms/step - loss: 4.4837 - accuracy: 0.0438 - val_loss: 4.5069 - val_accuracy: 0.0434
Epoch 4/25
413/413 [==============================] - 87s 208ms/step - loss: 4.4638 - accuracy: 0.0447 - val_loss: 4.4463 - val_accuracy: 0.0416
Epoch 5/25
413/413 [==============================] - 91s 218ms/step - loss: 4.4451 - accuracy: 0.0443 - val_loss: 4.4643 - val_accuracy: 0.0316
Epoch 6/25
413/413 [==============================] - 88s 210ms/step - loss: 4.4294 - accuracy: 0.0402 - val_loss: 4.4663 - val_accuracy: 0.0403
Epoch 7/25
413/413 [==============================] - 87s 207ms/step - loss: 4.4240 - accuracy: 0.0439 - val_loss: 4.5117 - val_accuracy: 0.0

In [ ]:
tf.keras.utils.plot_model(mobilenet_model, to_file='model.png', show_shapes=True, show_dtype=False,show_layer_names=True, rankdir='TB', expand_nested=False, dpi=72,layer_range=None, show_layer_activations=False)

In [ ]:
mobilenet_model.save_weights('model1.h5')
wandb.save('model1.h5')
wandb.finish()

accuracy,▁▄▅▅▆▆▆▆▇▆▇▆█▇██▇
epoch,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
loss,█▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▃█▅▃▄▃▄▂▄▄▄▃▂▅▂
val_loss,▆▃▃▂▃▃▇▄▁▅▅▅▆▇█▅▆
accuracy,0.10854
best_epoch,8
best_val_loss,3.99072
epoch,16
loss,3.83884
val_accuracy,0.09491


In [ ]:
loss, accuracy = mobilenet_model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

55/55 [==============================] - 7s 99ms/step - loss: 4.0852 - accuracy: 0.0953
Test accuracy : 0.09526526182889938


In [ ]:
# Try reloading the model
best_model = wandb.restore('model1.h5', run_path="Wenuka_19/DermAI-01/2fn4ukmr")
mobilenet_model.load_weights(best_model.name)

ValueError: ignored

In [ ]:
labels = []
predictions = []

for x, y in test_dataset:
  predictions = np.concatenate([predictions, np.argmax(mobilenet_model.predict(x), axis = -1)])
  labels = np.concatenate([labels, np.argmax(y.numpy(), axis=-1)])

In [ ]:
for i in range(len(labels)):
  print(class_names[int(labels[i])]," # ",class_names[int(predictions[i])])

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

accuracy = accuracy_score(labels, predictions)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(labels, predictions,average='macro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(labels, predictions,average='macro')
print('Recall: %f' % recall)

Accuracy: 0.124929
Precision: 0.052231
Recall: 0.070104


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from tensorflow import lite
converter = lite.TFLiteConverter.from_keras_model(mobilenet_model)
 
tfmodel = converter.convert()
 
open('iris.tflite', 'wb').write(tfmodel)

19332412